# Applicaiton will do two main steps
## 1. adjust total weight == Airway Bill GW
## 2. adjust 42/61/62 HS CODE item weight

###  initial version created on 10/01/2019


###  version 2   25/01/2019, changed some weight calculation methods
###  version 3   28/01/2019, changed according to final template, highlight the cell which value is > 35

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings import Range

In [3]:
wb = xw.Book(r'C:\Users\Xiaodong\Desktop\包税程序设计\重量计算流程及相关文件\重量计算流程及相关文件\模板\包税箱单发票模板.xlsx')  #open the original data file

# Below step maybe not necessary, just to creat a summary sheet with some weight information, ideally pre defined mannually

In [55]:
#'''
xw.sheets.add(name='Summary', before=wb.sheets[0])
wb.sheets[0].range('A1').value = 'Air Way Bill GW'
wb.sheets[0].range('A2').value = '包税 GW'
wb.sheets[0].range('A3').value = '客人自有税号 GW'
wb.sheets[0].range('A4').value = '包税修改后 GW'
wb.sheets[0].range('A1:A4').color = (0, 225, 0)
#'''

# assign the source sheets with original data

In [59]:
shtA = wb.sheets['A']    #worksheet A  wb.sheets[2]
sht_summary = wb.sheets['Summary']  #worksheet summary wb.sheets[0]

# Creat a new workbook to output the results

In [57]:
wb2 = xw.Book()

sht1 = wb2.sheets.active   # create a new workbook  'Book3 in current case'

# Export to DataFrame

#### method 2, calculate percentage of calculated weight and Air Waybill G.W.  This is prefered method

In [60]:
df = shtA.range('A2:U143').options(pd.DataFrame).value   # export to dataframe change A1: U143 according to individual case

df['CALCULATED GW'] = df['QTY'] * df['SHIIPING WEIGHT']  # create a column 'calculated gw' based on qty * SHIIPING WEIGHT

perc = round(sht_summary.range('B1').value / df['CALCULATED GW'].sum(), 2)

df['ADJUSTED GW'] = round(df['CALCULATED GW'] * perc, 2)

diff = round(df['ADJUSTED GW'].sum() - sht_summary.range('B1').value, 2)

# Find the item only appear once, apply diff to that item's weight

In [62]:
df['FULL DESCRICTION OF GOODS'].value_counts(ascending=True).index[0]  #find the items appearance, pick the first one by add .index[0] 

# df.loc[df['英文品名'] == df['英文品名'].value_counts(ascending=True).index[0]]['ADJUSTED GW'] + diff

df.loc[df['FULL DESCRICTION OF GOODS'] == df['FULL DESCRICTION OF GOODS'].value_counts(ascending=True).index[0], 'ADJUSTED GW'] -= diff

# step 1 finished up to here
# step 2 begins...

In [64]:
df['HS CODE'] = df['HS CODE'].astype(str)   # convert column HS CODE from float to string

def check_HS(x):
    if x.startswith('42') | x.startswith('61') | x.startswith('62'):
        return 'Sensitive'
    else:
        return 'Safe'

df['SENSITIVITY']= df['HS CODE'].apply(check_HS)  # create a column to set sensitivity according to HS CODE

## define a function to decide adjustment value according to qty

In [65]:
def weight_setter(qty):     #changed values according to new version
    if qty <=50:
        return 0.15
    elif qty <= 100:
        return 0.075
    elif qty <=150:
        return 0.05
    elif qty <=200:
        return 0.04
    else:
        return 0.03

# adjust sensitive HS CODE weight

In [66]:
def adjust_wt_sense(row):
    # weight = row[10]
    sens = row[22]
    qty = row[8]
    
    if sens == 'Sensitive':
        return qty * weight_setter(qty)

df['ADJ1_SENS'] = df.apply(adjust_wt_sense, axis = 1)

df['ADJ1'] = df['ADJ1_SENS'].combine_first(df['ADJUSTED GW'])   # replace the none value with original number

# adjust safe HS CODE weight


### Method 2, added on 25/01/2019, prefered method, this should elminate the negative weight issue

In [67]:
wgt_diff = round(sht_summary.range('B1').value - df['ADJ1'].sum(), 2)

df_safe = df[df['SENSITIVITY'] == 'Safe']

wgt_safe_bf = round(df_safe['ADJ1'].sum(), 2)

wgt_safe_af = round(wgt_safe_bf + wgt_diff, 2)

perc2 = round(wgt_safe_af / wgt_safe_bf, 2)

df['ADJ2_SAFE'] = df.apply(lambda row: round(row['ADJ1'] * perc2, 2) if row['SENSITIVITY'] == 'Safe' else np.nan, axis = 1)

df['ADJ2'] = df['ADJ2_SAFE'].combine_first(df['ADJ1']) 

#  Same again, find the items only appear once, apply final diff to that item's weight

In [75]:
final_diff = round(df['ADJUSTED GW'].sum() - df['ADJ2'].sum(), 2)

# final_diff

df['FULL DESCRICTION OF GOODS'].value_counts(ascending=True).index[1]  #find the items appearance, pick the first one by add .index[0] 

df.loc[df['FULL DESCRICTION OF GOODS'] == df['FULL DESCRICTION OF GOODS'].value_counts(ascending=True).index[1], 'ADJ2'] += final_diff

# upto to here, step two finished

# Conditonal Formatting, high light cell with value > 35

In [ ]:
# Let's say we have a column containing numbers, highlight the background color if number is > 30
for x in Range('AB2').expand('down'):   # change 'AB2' when to reflect the column
    if x.value > 30:
        x.color = (255,153,255)  # set background color to pink

# experimenting code below, mainly for test and check values of certain variable

In [ ]:
df.columns[22]

In [ ]:
df.columns[-1]

In [18]:
df['英文品名'].value_counts(ascending=True)

Agitator                     1
Stainless steel needle       1
Hanger                       1
humidifier                   1
Eyelash brush                1
Finger band                  1
strap                        1
belt                         1
Toe separation sleeve        1
LED light                    1
Arm guard                    1
Finger                       1
Vest                         1
Shift knob                   1
wrench                       1
Foot protection              1
nflator                      1
Bracers                      1
gloves                       1
Table bar chair yarn         1
Clip                         1
Exhaust gas cover            1
Air conditioning panel       1
Foot cover                   1
Mask                         1
plastic cup                  1
handlebar                    1
exhaust pipe                 1
Drying rack                  1
Shoehorn                     1
Sling                        1
handle                       1
Speaker 

In [60]:
df_test =df

In [63]:
df_test['SENSITIVITY'] = df_test['SENSITIVITY'].astype('category')

In [ ]:
df_test['adj3_safe'] = df_test.apply()

In [65]:
mask2 = df_test['SENSITIVITY'] == 'Safe'

# write back to worksheet, execute below code whenever necessary

In [76]:
sht1.range('A1').value = df

# clear worksheet content

In [69]:
sht1.clear_contents()